In [ ]:
import random
import math


def read_bpp_file(file_path):
    with open(file_path, 'r') as f:
        n = int(f.readline().strip())
        capacity = int(f.readline().strip())
        objects = [int(line.strip()) for line in f]
    return (n, capacity, objects)


(n , bin_capacity , items ) = read_bpp_file('BPP_50_50_0.1_0.7_0.txt')
# (n , bin_capacity , items ) = read_bpp_file('Hard28_BPP900.txt')
# random.shuffle(items)

In [ ]:
class Node:
    def __init__(self, level: int, bins: list[list[int]]):
        self.level = level  # depth in the tree
        self.bins = bins  # list of configuration of objects (list of bins: each bin is a list of objects in it)
        self.children = []  # list of child nodes

    def __str__(self):
        return f"{self.bins}"  #  + "->>" + "   ".join(map(str, self.children))


def generer_fils(list_objects: list[int], capacity_bins: int, node: Node):
    i = node.level  # get current level of tree
    obj = list_objects[i]  # get current object to be added to bins

    n2 = Node(i + 1, node.bins + [[obj]])  # we add it to a new bin
    node.children.append(n2)
    for j, bin_ in enumerate(node.bins):   # for each bin of the node
        if sum(bin_) + obj <= capacity_bins:  # if we can add it to the current bin
            n1 = Node(i + 1, node.bins.copy())
            n1.bins[j] = bin_ + [obj]
            node.children.append(n1)
    
    return node.children


def evaluer(node: Node):
    if node.level < n - 1:  # if noeud interne
        #return len(node.bins) + math.ceil((sum(items[node.level + 1:]) - sum(bin_capacity - sum(node.bins[x]) for x in range(len(node.bins)) if bin_capacity - sum(node.bins[x]) > items[-1])) / bin_capacity)
        # current nb of bins + (total capacity of objects left - leftover capacity in all bins) / capacity
        
        #return len(node.bins) + math.ceil((sum(items[node.level + 1:]) - (bin_capacity - sum(node.bins[-1]))) / bin_capacity)
        # current nb of bins + (total capacity of objects left - leftover capacity in the last bin) / capacity
        # to estimate the extra nb of bins that this config will have
        
        return len(node.bins) + math.ceil((sum(items[node.level + 1:])) / bin_capacity)
        # current nb of bins + total capacity of objects left / capacity
        
        #return len(node.bins) + n - node.level
    return len(node.bins)  # if les feuilles, return nb bins


def branch_and_bound(list_objects: list[int], capacity_bins: int):
    root = Node(0, [])
    actif_nodes = generer_fils(list_objects, capacity_bins, root)
    borne_sup = len(list_objects) + 1  # borne sup of nb bins (+inf)
    best_sol = None
    while actif_nodes:
        node = actif_nodes.pop()  # parcours en profondeur
        #print(node.level)
        if node.level < len(list_objects) - 1:  # not feuille
            #print("Node evaluation", evaluer(node))
            if evaluer(node) <= borne_sup:  # if it's has potential, we don't prune the node
                actif_nodes.extend(generer_fils(list_objects, capacity_bins, node))
                """children = generer_fils(list_objects, capacity_bins, node)
                for child in children:
                    if evaluer(child) <= borne_sup:  # if it's has potential, we don't prune the child
                        actif_nodes.append(child)"""
        else:  # feuille
            #print(node.level)
            evaluation = evaluer(node)  # calc its cost
            if evaluation <= borne_sup:  # it's better than current solution
                borne_sup = evaluation
                best_sol = node
                # print("New evaluation", borne_sup)
    return best_sol


r = branch_and_bound(items, bin_capacity)

# solution
print(f"solution cost {len(r.bins)}, solution:", r)


solution cost 23, solution: [[35, 7, 7], [35, 10], [34, 9, 7], [34, 9, 7], [34, 10], [33, 11], [33, 11], [33, 12], [32, 13], [31, 13], [31, 13], [30, 16], [30, 16], [29, 16], [28, 16], [28, 16], [28, 18], [27, 18], [26, 20], [26, 21], [26, 22], [26, 22], [25, 25]]


# New Section

OLD WORK NOT WORKING

In [ ]:
"""
from queue import PriorityQueue


def bin_packing(items, bin_size):
    # Sort the items in decreasing order of their sizes
    sorted_items = sorted(items, reverse=True)

    # Set the initial upper bound to infinity
    ub = 1000000

    # Initialize the root node with an empty bin and all items
    root_node = (0, [], sorted_items)

    # Create a priority queue and insert the root node
    queue = PriorityQueue()
    queue.put((-ub, root_node))

    # Loop until the queue is empty
    while not queue.empty():
        # Get the node with the highest priority
        _, node = queue.get()

        # Unpack the node
        lb, bins, remaining_items = node
        # If there are no remaining items, update the upper bound
        if not remaining_items:
            ub = min(ub, len(bins))
            if (ub <= len(bins)): save_bins = bins
            continue

        # If the lower bound is already greater than or equal to the current upper bound, prune the node
        if lb >= ub:
            continue

        # Create two child nodes: one where the next item is packed in a new bin, and one where it is packed in an existing bin
        new_bin = bins + [remaining_items[:1]]
        new_items = remaining_items[1:]
        new_lb = max(lb, len(new_bin))
        queue.put((-new_lb, (new_lb, new_bin, new_items)))

        for i, bin in enumerate(bins):
            if sum(bin) + remaining_items[0] <= bin_size:
                new_bin = bins[:i] + [bin + [remaining_items[0]]] + bins[i+1:]
                new_items = remaining_items[1:]
                new_lb = max(lb, len(new_bin))
                queue.put((-new_lb, (new_lb, new_bin, new_items)))
    # Return the optimal number of bins
    return ub, save_bins


nb_bins, bins = bin_packing(items, bin_capacity)

print(nb_bins)
print(bins)"""

'\nfrom queue import PriorityQueue\n\n\ndef bin_packing(items, bin_size):\n    # Sort the items in decreasing order of their sizes\n    sorted_items = sorted(items, reverse=True)\n\n    # Set the initial upper bound to infinity\n    ub = 1000000\n\n    # Initialize the root node with an empty bin and all items\n    root_node = (0, [], sorted_items)\n\n    # Create a priority queue and insert the root node\n    queue = PriorityQueue()\n    queue.put((-ub, root_node))\n\n    # Loop until the queue is empty\n    while not queue.empty():\n        # Get the node with the highest priority\n        _, node = queue.get()\n\n        # Unpack the node\n        lb, bins, remaining_items = node\n        # If there are no remaining items, update the upper bound\n        if not remaining_items:\n            ub = min(ub, len(bins))\n            if (ub <= len(bins)): save_bins = bins\n            continue\n\n        # If the lower bound is already greater than or equal to the current upper bound, prun

In [ ]:
'''import random
import math

class BPPInstance:
    def __init__(self, n, capacity, objects):
        self.n = n
        self.capacity = capacity
        self.objects = objects


class Node:
    def __init__(self, level: int, bins: list[list[int]]):
        self.level = level  # depth in the tree
        self.bins = bins  # list of configuration of objects (list of bins: each bin is a list of objects in it)
        self.children = []  # list of child nodes

    def __str__(self):
        return f"{self.level}: {self.bins}" + "\n" + "   ".join(map(str, self.children))

def read_bpp_file(file_path):
    with open(file_path, 'r') as f:
        n = int(f.readline().strip())
        capacity = int(f.readline().strip())
        objects = [int(line.strip()) for line in f]
    return BPPInstance(n, capacity, objects)

def generer_espace_solution(list_objects: list[int], capacity_bins: int):
    root = Node(0, [])
    parents = []
    for i, obj in enumerate(list_objects):
        if i == 0:
            root.bins.append([obj])
            parents = [root]  # depth queue
        else:
            new_parents = []  # stock next depth queue
            for parent in parents:
                for j, bin_ in enumerate(parent.bins):   # for each bin
                    if sum(bin_) + obj <= capacity_bins:  # if we can add it to the current bin
                        n1 = Node(i, parent.bins.copy())
                        n1.bins[j] = bin_.copy() + [obj]
                        parent.children.append(n1)
                        new_parents.append(n1)
                n2 = Node(i, parent.bins.copy() + [[obj]])  # we add it to a new bin: new bin
                parent.children.append(n2)
                new_parents.append(n2)
            parents = new_parents
    return root


def evaluer(node: Node):
    if node.level < n - 1:
        # if noeud interne
        return len(node.bins) + math.ceil((sum(objects[node.level + 1:]) - (capacity - sum(node.bins[-1]))) / capacity)
        # current nb of bins + (total capacity of objects left - leftover capacity in the last bin) / capacity
        # to estimate the extra nb of bins that this config will have
    return len(node.bins)  # if les feuilles, return nb bins


def branch_and_bound(root: Node):
    while len(root.children) == 1:  # we ignore the first levels where we only have one child: no branches to explore
        root = root.children[0]
    actif_nodes = [root]
    borne_sup = n + 1  # borne sup of nb bins (+inf)
    best_sol = None
    while actif_nodes:
        node = actif_nodes.pop(0)
        children = node.children
        for child in children:
            evaluation = evaluer(child)
            if evaluation <= borne_sup:  # we found a node that contain solution
                borne_sup = evaluation
                best_sol = child
            if child.children:  # not feuille
                actif_nodes.append(child)
    return best_sol


r = generer_espace_solution(items, bin_capacity)
# affichage arbre
"""l = [r]
while l:
    print(l[0].level, "   ".join([f"{no.bins}{len(no.children)} {evaluer(no)};" for no in l]))
    l2 = []
    for no in l:
        l2.extend(no.children)
    l = l2"""
# solution
#print("solution: ", branch_and_bound(r))
import time

start_time = time.perf_counter()
solution = branch_and_bound(r)
end_time = time.perf_counter()
print("solution: ", branch_and_bound(r))
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
for i, bin_ in enumerate(solution.bins):
    ax.bar(i, sum(bin_), label=f"Bin {i+1}")
    for j, obj in enumerate(bin_):
        ax.text(i, j+obj/2, f"{obj}", ha="center", va="center")
ax.set_xlabel("Bins")
ax.set_ylabel("Capacity")
ax.set_title(f"Bins and objects at level {solution.level}")
ax.legend()
ax.text(0.05, 0.95, f"Execution time: {(end_time - start_time)*1000:.3f} ms", transform=ax.transAxes, fontsize=10,
            verticalalignment='top')
plt.show()
'''

'import random\nimport math\n\nclass BPPInstance:\n    def __init__(self, n, capacity, objects):\n        self.n = n\n        self.capacity = capacity\n        self.objects = objects\n\n\nclass Node:\n    def __init__(self, level: int, bins: list[list[int]]):\n        self.level = level  # depth in the tree\n        self.bins = bins  # list of configuration of objects (list of bins: each bin is a list of objects in it)\n        self.children = []  # list of child nodes\n\n    def __str__(self):\n        return f"{self.level}: {self.bins}" + "\n" + "   ".join(map(str, self.children))\n\ndef read_bpp_file(file_path):\n    with open(file_path, \'r\') as f:\n        n = int(f.readline().strip())\n        capacity = int(f.readline().strip())\n        objects = [int(line.strip()) for line in f]\n    return BPPInstance(n, capacity, objects)\n\ndef generer_espace_solution(list_objects: list[int], capacity_bins: int):\n    root = Node(0, [])\n    parents = []\n    for i, obj in enumerate(list_o

In [ ]:
'''import random
import math


class Node:
    def __init__(self, level: int, bins: list[list[int]]):
        self.level = level  # depth in the tree
        self.bins = bins  # list of configuration of objects (list of bins: each bin is a list of objects in it)
        self.children = []  # list of child nodes

    def __str__(self):
        return f"{self.level}: {self.bins}" + "\n" + "   ".join(map(str, self.children))


def generer_espace_solution(list_objects: list[int], capacity_bins: int):
    root = Node(0, [])
    parents = []
    for i, obj in enumerate(list_objects):
        if i == 0:
            root.bins.append([obj])
            parents = [root]  # depth queue
        else:
            new_parents = []  # stock next depth queue
            for parent in parents:
                for j, bin_ in enumerate(parent.bins):   # for each bin
                    if sum(bin_) + obj <= capacity_bins:  # if we can add it to the current bin
                        n1 = Node(i, parent.bins.copy())
                        n1.bins[j] = bin_.copy() + [obj]
                        parent.children.append(n1)
                        new_parents.append(n1)
                n2 = Node(i, parent.bins.copy() + [[obj]])  # we add it to a new bin: new bin
                parent.children.append(n2)
                new_parents.append(n2)
            parents = new_parents
    return root


def evaluer(node: Node):
    if node.level < n - 1:
        # if noeud interne
        return len(node.bins) + math.ceil((sum(objects[node.level + 1:]) - (capacity - sum(node.bins[-1]))) / capacity)
        # current nb of bins + (total capacity of objects left - leftover capacity in the last bin) / capacity
        # to estimate the extra nb of bins that this config will have
    return len(node.bins)  # if les feuilles, return nb bins


def branch_and_bound(root: Node, sup : int):
    while len(root.children) == 1:  # we ignore the first levels where we only have one child: no branches to explore
        root = root.children[0]
    actif_nodes = [root]
    borne_sup = sup # borne sup of nb bins (+inf)
    best_sol = None
    while actif_nodes:
        node = actif_nodes.pop(0)
        children = node.children
        for child in children:
            evaluation = evaluer(child)
            if evaluation <= borne_sup:  # we found a node that contain solution
                borne_sup = evaluation
                best_sol = child
                if child.children:  # not feuille
                    actif_nodes.append(child)
    return best_sol



n = 18  # nb objects
capacity = 20  # capacity of bins
objects = sorted([random.randint(1, capacity) for _ in range(n)], reverse=True)  # generate an instance of the problem (sorted)
print(objects)
borne=objects[0]*len(objects)/capacity
r = generer_espace_solution(objects, capacity)
# affichage arbre
"""l = [r]
while l:
    print(l[0].level, "   ".join([f"{no.bins}{len(no.children)} {evaluer(no)};" for no in l]))
    l2 = []
    for no in l:
        l2.extend(no.children)
    l = l2"""
# solution
R=branch_and_bound(r,borne)
print("solution: ", R)
print(len(R.nb_bins))'''

'import random\nimport math\n\n\nclass Node:\n    def __init__(self, level: int, bins: list[list[int]]):\n        self.level = level  # depth in the tree\n        self.bins = bins  # list of configuration of objects (list of bins: each bin is a list of objects in it)\n        self.children = []  # list of child nodes\n\n    def __str__(self):\n        return f"{self.level}: {self.bins}" + "\n" + "   ".join(map(str, self.children))\n\n\ndef generer_espace_solution(list_objects: list[int], capacity_bins: int):\n    root = Node(0, [])\n    parents = []\n    for i, obj in enumerate(list_objects):\n        if i == 0:\n            root.bins.append([obj])\n            parents = [root]  # depth queue\n        else:\n            new_parents = []  # stock next depth queue\n            for parent in parents:\n                for j, bin_ in enumerate(parent.bins):   # for each bin\n                    if sum(bin_) + obj <= capacity_bins:  # if we can add it to the current bin\n                    

In [ ]:
class Node:
    def __init__(self, wRemaining, level, numBoxes):   
        self.wRemaining = wRemaining    #array of remaining weight for each box 
        self.level = level              #the level of the node in branch and bound tree 
        self.numBoxes = numBoxes        #number of used boxes  

    def getLevel(self):        
        return self.level

    def getNumberBoxes(self):                              
        return self.numBoxes

    def getWRemainings(self):                              
        return self.wRemaining

    def getWRemaining(self, i):                             
        return self.wRemaining[i]


#branch and bound bin packing exact algorithm
def branchAndBound(n, c, w):

    minBoxes = n    #initialize the number of used boxes 
    Nodes = []      #array contains the unprocessed nodes
    wRemaining = [c]*n  #array contains remaining weight in each box [c,c,c,.......c]
    numBoxes = 0    #initialise number of used boxes 
    best_sol = None

    curN = Node(wRemaining, 0, numBoxes)    #create the root node, level 0, number of used boxes 0

    Nodes.append(curN)  #add the root node to Nodes

    while len(Nodes) > 0 :  #As long as Nodes is not empty do 

        curN = Nodes.pop()      #get a node to process it : curent node (curN)
        curLevel = curN.getLevel()  #get the level of curN

        if (curLevel == n) and (curN.getNumberBoxes() < minBoxes):  #if this node is a leaf of the tree and the number of used boxes < minBoxes
            best_sol = curN
            minBoxes = curN.getNumberBoxes()    #update minBoxes

        else:
            indNewBox = curN.getNumberBoxes()   

            if (indNewBox < minBoxes ):     #else, if from this node the number of used boxes < minBoxes (not dominated)

                wCurLevel = w[curLevel]     
                for i in range(indNewBox+1):    #we will try to add the following item in each box already used, and in a new box (that's why I added +1 to indNewBox)
                        
                    if  (curLevel < n) and (curN.getWRemaining(i) >= wCurLevel):    #if it is possible to add the item in the box i (weight remaining in the box > weight of the item)
                                                                                    #we will create a node and add it to Nodes.
                        newWRemaining = curN.getWRemainings().copy()
                        newWRemaining[i] -= wCurLevel                       #remaining weight in box i - weight of the item to be added

                        if (i == indNewBox):                                            #new Box 
                            newNode = Node(newWRemaining, curLevel + 1, indNewBox + 1)
                        else:                                                           #already used box
                            newNode = Node(newWRemaining, curLevel + 1, indNewBox)
                        
                        Nodes.append(newNode)
        
    return minBoxes, best_sol


l, r = branchAndBound(n, bin_capacity, items)

# solution
print(f"solution cost", l, "solution:", r.wRemaining)

solution cost 23 solution: [1, 5, 0, 0, 6, 6, 6, 5, 5, 6, 6, 4, 4, 5, 6, 6, 4, 0, 4, 3, 2, 2, 0, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]


In [ ]:
import heapq
import time
import heapq


class Item(object):
    def __init__(self, name, weight):

        self._name = name
        self._weight = weight

    @property
    def name(self):


        return self._name

    @property
    def weight(self):


        return self._weight

    def __lt__(self, other):


        return self._weight < other._weight

    def __cmp__(self, other):


        return self._weight < other._weight

    def __eq__(self, other):


        return self._weight == other._weight


class Bin(object):
    def __init__(self, capacity):

        self._items = []
        self._utilization = 0
        self._capacity = capacity

    @property
    def name(self):


        return self._name

    @name.setter
    def name(self, name):

        self._name = name

    @property
    def capacity(self):


        return self._capacity

    @property
    def total_weight(self):


        return sum([i for i in self._items])

    @property
    def utilization(self):

        total_weight = sum([i for i in self._items])
        return round((total_weight / self._capacity) * 100, 2)


    def push(self, item):

        heapq.heappush(self._items, item)

    def pop(self):

        return heapq.heappop(self._items)

    def remove(self, name):

        index = 0
        for i, item in enumerate(self._items):
            index = i
            if item.name == name:
                break
        del self._items[index]
        heapq.heapify(self._items)


    def get_items(self, attribute=None, value=None):

        if attribute and value:
            return [i for i in self._items if getattr(i, attribute) == value]
        return self._items


class Binpacker(object):
    def __init__(self, capacity):

        self._capacity = capacity
        self._bins = []
        self._items = []

    @property
    def items(self):


        return self._items

    @items.setter
    def items(self, items):


        for i in items:
            if i > self._capacity:
                raise Exception(
                    'Weight of Item: "{}" exceeds the maximum '
                    'bin capacity: {}.'.format(
                        i.name, self._capacity))
        self._items = items

    @property
    def bins(self):

        return self._bins

    @bins.setter
    def bins(self, bins):

        self._bins = bins

    def get_truth_table(self, capacity, items):

        m = [
            [True for j in range(capacity + 1)]
            for i in range(len(items))
        ]
        for index, item in enumerate(items, 1):
            i = index - 1
            for j in range(0, capacity + 1):
                if not i:
                    if j != item and j > 0:
                        m[i][j] = False
                else:
                    if j < item:
                        m[i][j] = m[i - 1][j]
                    else:
                        m[i][j] = m[i - 1][j] or m[i - 1][j - item]
        return m

    def _pick_items(self, truth_table):
        k = len(truth_table) - 1
        picked_items_indices = []
        if k >= 0:
            # Get the heaviest subtotal weight (best utilization)
            # a bin can be loaded.
            j = max([index for index, x in enumerate(truth_table[k]) if x])

            while k >= 0:
                if not k:
                    if j > 0:
                        picked_items_indices.append(k)
                        # old_bin.push(self._items[k])
                        # del self._items[k]
                else:
                    if not truth_table[k - 1][j]:
                        picked_items_indices.append(k)
                        # old_bin.push(self._items[k])
                        # del self._items[k]
                        j -= self._items[k]
                k -= 1
        return picked_items_indices

    def _move_items_to_bin(self, list_of_items_indices, bin_index):

        for i in list_of_items_indices:
            self._bins[bin_index].push(self._items[i])
            del self._items[i]

    def pack_items(self):

        self._items = sorted(self._items)
        # First, go through existing bin to see if they can
        # accommodate new weights.
        for index, old_bin in enumerate(self._bins):
            if old_bin.utilization == 100.00:
                continue
            remaining_space = old_bin.capacity - old_bin.total_weight
            m = self.get_truth_table(remaining_space, self._items)
            picked_items = self._pick_items(m)
            self._move_items_to_bin(picked_items, index)

        # If there is still remaining Items, we need to get a new Bin.
        while len(self._items) > 0:
            m = self.get_truth_table(self._capacity, self._items)
            new_bin = Bin(self._capacity)
            new_bin.name = '[NEW BIN {}]'.format(len(self._bins))
            self._bins.append(new_bin)
            bin_index = len(self._bins) - 1
            picked_items = self._pick_items(m)
            self._move_items_to_bin(picked_items, bin_index)
def read_bpp_file(file_path):
    with open(file_path, 'r') as f:
        cap = int(f.readline().strip())
        n = int(f.readline().strip())
        capacity = int(f.readline().strip())
        objects = [int(line.strip()) for line in f]
    return (n, capacity, objects , cap)
(n , capacity , objects , cap ) = read_bpp_file('BPP_50_50_0.1_0.7_0.txt')    

def main():
    print("Capacite", capacity)
    print("Num Objets " , n )
    print("Objets: ", objects  )
    packer = Binpacker(capacity)
    packer.items = objects


    j=0
    while True:
        packer.pack_items()
        for i, x in enumerate(packer.bins):
            if (i==cap):
                break
            else :
                print('Bin ({}%) {}: {}'.format(x.utilization, i,[i for i in x.get_items()]))

        break

if __name__ == "__main__":
    main()